In [1]:
import theano
import pickle
import sys
import os
sys.path.append('../..')
import numpy as np
import pandas as pd
import gru4rec #If this shows an error probably the notebook is not in GRU4Rec/examples/rsc15/
import evaluation
from IPython.core.debugger import set_trace
import time
import datetime

In [2]:
def timestamp():
    return datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S') + ' '

In [3]:
PATH_TO_TRAIN = '/Users/sx5640/Code/jupyter/GRU4Rec/examples/rsc15/output/rsc15_train_full.txt'
PATH_TO_TEST = '/Users/sx5640/Code/jupyter/GRU4Rec/examples/rsc15/output/rsc15_test.txt'

data = pd.read_csv(PATH_TO_TRAIN, sep='\t', dtype={'ItemId':np.int64})
valid = pd.read_csv(PATH_TO_TEST, sep='\t', dtype={'ItemId':np.int64})

In [4]:
print('unique session ids: ' + str(data['SessionId'].unique().size))
print('full data size: ' + str(data.size))
print('validation data size: ' + str(valid.size))

unique session ids: 7966257
full data size: 94911717
validation data size: 213666


In [5]:
sample_session_size = 100000
data_sample = data[data['SessionId'].isin(np.random.choice(data['SessionId'].unique(),sample_session_size))]
valid_sample = valid[valid['ItemId'].isin(data_sample['ItemId'].unique())]
print('sample size: ' + str(data_sample.size))
print('sample ratio :' + str(data_sample.size/data.size))
print('validation data size: ' + str(valid_sample.size))

sample size: 1180704
sample ratio :0.0124400236064
validation data size: 206385


### Test Run

# Grid Search

### loss hidden_act final_act

In [6]:
grid_search_params = {
    'loss': ['top1', 'bpr', 'cross-entropy', 'xe_logit', 'top1-max', 'bpr-max-0.001', 'bpr-max-0.01', 'bpr-max-0.1','bpr-max-1','bpr-max-10'],
    'hidden_act' : ['tanh', 'relu', 'linear'],
    'final_act' : ['tanh', 'softmax', 'linear', 'relu', 'softmax_logit', 'leaky-0.001', 'leaky-0.01', 'leaky-0.1', 'elu-1', 'elu-0.1']
}
n_layers = 100
default_params = {
    'layers': [n_layers], 
    'loss': 'bpr-max-2', 
    'batch_size': 50, 
    'final_act': 'tanh', 
    'hidden_act': 'tanh',
    'dropout_p_hidden': 0.5, 
    "learning_rate": 0.01, 
    "momentum": 0.0
}

results = []

In [7]:
for loss in grid_search_params['loss']:
    for hidden_act in grid_search_params['hidden_act']:
        for final_act in grid_search_params['final_act']:
            params = default_params.copy()
            config = {
                'loss': loss,
                'hidden_act': hidden_act,
                'final_act': final_act,
            }
            params.update(config)
            print(timestamp() + 'Starting training with configuration: ' + str(config))
            try:
                gru = gru4rec.GRU4Rec(**params)
                gru.fit(data_sample)
                v_sample = valid_sample.copy()
                res = evaluation.evaluate_sessions_batch(gru, v_sample, None,cut_off=20)
                print(timestamp() + 'Recall: ' + str(res[0]) + ' MRR: ' + str(res[1]))
                results += [{
                    'params': config,
                    'results': res
                }]
            except Exception as e:
                results += [{
                    'params': config,
                    'failed': True,
                    'error_during_train': gru.error_during_train
                }]
                print(timestamp() + 'Training/Evaluation failed')

2017-09-20 18:06:11 Starting training with configuration: {'loss': 'top1', 'hidden_act': 'tanh', 'final_act': 'tanh'}
2017-09-20 18:06:21 Epoch0	loss: 0.984365
2017-09-20 18:06:29 Epoch1	loss: 0.968452
2017-09-20 18:06:37 Epoch2	loss: 0.955128
2017-09-20 18:06:45 Epoch3	loss: 0.944128
2017-09-20 18:06:54 Epoch4	loss: 0.934939
2017-09-20 18:07:02 Epoch5	loss: 0.926991
2017-09-20 18:07:10 Epoch6	loss: 0.920349
2017-09-20 18:07:18 Epoch7	loss: 0.914342
2017-09-20 18:07:26 Epoch8	loss: 0.909236
2017-09-20 18:07:34 Epoch9	loss: 0.904461
Measuring Recall@19 and MRR@19


../../evaluation.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  test_data.sort_values([session_key, time_key], inplace=True)


2017-09-20 18:07:51 Recall: 0.187175561116MRR: 0.0827112861949
2017-09-20 18:07:51 Starting training with configuration: {'loss': 'top1', 'hidden_act': 'tanh', 'final_act': 'softmax'}
2017-09-20 18:08:01 Epoch0	loss: 0.990098
2017-09-20 18:08:09 Epoch1	loss: 0.990097
2017-09-20 18:08:17 Epoch2	loss: 0.990097
2017-09-20 18:08:25 Epoch3	loss: 0.990095
2017-09-20 18:08:33 Epoch4	loss: 0.990083
2017-09-20 18:08:41 Epoch5	loss: 0.989965
2017-09-20 18:08:49 Epoch6	loss: 0.989871
2017-09-20 18:08:57 Epoch7	loss: 0.989836
2017-09-20 18:09:05 Epoch8	loss: 0.989819
2017-09-20 18:09:13 Epoch9	loss: 0.989806
Measuring Recall@19 and MRR@19


../../evaluation.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  test_data.sort_values([session_key, time_key], inplace=True)


2017-09-20 18:09:36 Recall: 0.00802927885872MRR: 0.0024920416172
2017-09-20 18:09:36 Starting training with configuration: {'loss': 'top1', 'hidden_act': 'tanh', 'final_act': 'linear'}
2017-09-20 18:09:47 Epoch0	loss: 0.983945
2017-09-20 18:09:54 Epoch1	loss: 0.966191
2017-09-20 18:10:02 Epoch2	loss: 0.949037
2017-09-20 18:10:10 Epoch3	loss: 0.935019
2017-09-20 18:10:18 Epoch4	loss: 0.922409
2017-09-20 18:10:25 Epoch5	loss: 0.911389
2017-09-20 18:10:33 Epoch6	loss: 0.901036
2017-09-20 18:10:41 Epoch7	loss: 0.891927
2017-09-20 18:10:49 Epoch8	loss: 0.883164
2017-09-20 18:10:57 Epoch9	loss: 0.875592
Measuring Recall@19 and MRR@19


...

2017-09-20 23:22:20 Recall: 0.383930238638MRR: 0.163946739827
2017-09-20 23:22:20 Starting training with configuration: {'loss': 'bpr-max-1', 'hidden_act': 'tanh', 'final_act': 'softmax'}
2017-09-20 23:22:31 Epoch0	loss: 0.693546
2017-09-20 23:22:40 Epoch1	loss: 0.693542
2017-09-20 23:22:49 Epoch2	loss: 0.693401
2017-09-20 23:22:58 Epoch3	loss: 0.693144
2017-09-20 23:23:07 Epoch4	loss: 0.693073
2017-09-20 23:23:16 Epoch5	loss: 0.693009
2017-09-20 23:23:25 Epoch6	loss: 0.692817
2017-09-20 23:23:34 Epoch7	loss: 0.692532
2017-09-20 23:23:44 Epoch8	loss: 0.692076
2017-09-20 23:23:53 Epoch9	loss: 0.691574
Measuring Recall@19 and MRR@19
2017-09-20 23:24:19 Recall: 0.00909362512604MRR: 0.00350369908422
2017-09-20 23:24:19 Starting training with configuration: {'loss': 'bpr-max-1', 'hidden_act': 'tanh', 'final_act': 'linear'}
2017-09-20 23:24:29 Epoch0	loss: 0.677775
2017-09-20 23:24:37 Epoch1	loss: 0.645324
2017-09-20 23:24:46 Epoch2	loss: 0.621409
2017-09-20 23:24:55 Epoch3	loss: 0.603739
20

2017-09-20 23:45:09 Epoch8	loss: 0.669249
2017-09-20 23:45:18 Epoch9	loss: 0.667526
Measuring Recall@19 and MRR@19
2017-09-20 23:45:30 Recall: 0.0605743735295MRR: 0.0422604550786
2017-09-20 23:45:30 Starting training with configuration: {'loss': 'bpr-max-1', 'hidden_act': 'relu', 'final_act': 'softmax_logit'}
2017-09-20 23:45:41 Epoch0	loss: 15.990174
2017-09-20 23:45:50 Epoch1	loss: 15.974898
2017-09-20 23:45:59 Epoch2	loss: 15.963835
2017-09-20 23:46:08 Epoch3	loss: 15.955972
2017-09-20 23:46:17 Epoch4	loss: 15.950101
2017-09-20 23:46:27 Epoch5	loss: 15.945407
2017-09-20 23:46:36 Epoch6	loss: 15.941484
2017-09-20 23:46:45 Epoch7	loss: 15.938092
2017-09-20 23:46:54 Epoch8	loss: 15.935266
2017-09-20 23:47:04 Epoch9	loss: 15.932650
Measuring Recall@19 and MRR@19
2017-09-20 23:47:30 Recall: 0.00048549127983MRR: 9.30437027893e-05
2017-09-20 23:47:30 Starting training with configuration: {'loss': 'bpr-max-1', 'hidden_act': 'relu', 'final_act': 'leaky-0.001'}
2017-09-20 23:47:40 Epoch0	loss

2017-09-21 00:07:14 Epoch3	loss: 11.657632
2017-09-21 00:07:23 Epoch4	loss: 72691.734375
2017-09-21 00:07:31 Epoch5	loss: 5148.583984
2017-09-21 00:07:40 Epoch6	loss: 418.997314
2017-09-21 00:07:48 Epoch7	loss: 90323591168.000000
2017-09-21 00:07:57 Epoch8	loss: 13189.384766
2017-09-21 00:08:05 Epoch9	loss: 10292.023438
Measuring Recall@19 and MRR@19
2017-09-21 00:08:16 Recall: 0.243249803936MRR: 0.103291196634
2017-09-21 00:08:16 Starting training with configuration: {'loss': 'bpr-max-1', 'hidden_act': 'linear', 'final_act': 'leaky-0.1'}
2017-09-21 00:08:25 Epoch0	loss: 0.693903
2017-09-21 00:08:34 Epoch1	loss: 78.319572
2017-09-21 00:08:42 Epoch2	loss: 0.651993
2017-09-21 00:08:51 Epoch3	loss: 304.906647
2017-09-21 00:08:59 Epoch4	loss: 0.615859
2017-09-21 00:09:08 Epoch5	loss: 0.604686
2017-09-21 00:09:16 Epoch6	loss: 0.598432
2017-09-21 00:09:25 Epoch7	loss: 1263.025879
2017-09-21 00:09:33 Epoch8	loss: 0.582629
2017-09-21 00:09:42 Epoch9	loss: 0.577691
Measuring Recall@19 and MRR@1

2017-09-21 00:29:40 Epoch0	loss: 0.688185
2017-09-21 00:29:49 Epoch1	loss: 0.676355
2017-09-21 00:29:58 Epoch2	loss: 0.666569
2017-09-21 00:30:08 Epoch3	loss: 0.658549
2017-09-21 00:30:17 Epoch4	loss: 0.651572
2017-09-21 00:30:26 Epoch5	loss: 0.645520
2017-09-21 00:30:35 Epoch6	loss: 0.639768
2017-09-21 00:30:44 Epoch7	loss: 0.634691
2017-09-21 00:30:53 Epoch8	loss: 0.630030
2017-09-21 00:31:02 Epoch9	loss: 0.625799
Measuring Recall@19 and MRR@19
2017-09-21 00:31:24 Recall: 0.346622101057MRR: 0.13778707606
2017-09-21 00:31:24 Starting training with configuration: {'loss': 'bpr-max-10', 'hidden_act': 'relu', 'final_act': 'tanh'}
2017-09-21 00:31:34 Epoch0	loss: 0.689175
2017-09-21 00:31:43 Epoch1	loss: 0.679254
2017-09-21 00:31:52 Epoch2	loss: 0.671809
2017-09-21 00:32:01 Epoch3	loss: 0.666388
2017-09-21 00:32:10 Epoch4	loss: 0.662289
2017-09-21 00:32:19 Epoch5	loss: 0.658978
2017-09-21 00:32:27 Epoch6	loss: 0.656329
2017-09-21 00:32:36 Epoch7	loss: 0.654012
2017-09-21 00:32:45 Epoch8	l

2017-09-21 00:52:49 Epoch0	loss: 0.684501
2017-09-21 00:52:57 Epoch1	loss: 0.670598
2017-09-21 00:53:06 Epoch2	loss: 0.661528
2017-09-21 00:53:14 Epoch3	loss: 0.654993
2017-09-21 00:53:22 Epoch4	loss: 0.649801
2017-09-21 00:53:31 Epoch5	loss: 0.645435
2017-09-21 00:53:39 Epoch6	loss: 0.641864
2017-09-21 00:53:47 Epoch7	loss: 0.638705
2017-09-21 00:53:55 Epoch8	loss: 0.635821
2017-09-21 00:54:04 Epoch9	loss: 0.633411
Measuring Recall@19 and MRR@19
2017-09-21 00:54:13 Recall: 0.367722298988MRR: 0.137168513813
2017-09-21 00:54:13 Starting training with configuration: {'loss': 'bpr-max-10', 'hidden_act': 'linear', 'final_act': 'relu'}
2017-09-21 00:54:23 Epoch0	loss: 0.691091
2017-09-21 00:54:31 Epoch1	loss: 0.685353
2017-09-21 00:54:39 Epoch2	loss: 0.680824
2017-09-21 00:54:48 Epoch3	loss: 0.677007
2017-09-21 00:54:56 Epoch4	loss: 0.673829
2017-09-21 00:55:04 Epoch5	loss: 0.670920
2017-09-21 00:55:13 Epoch6	loss: 0.668391
2017-09-21 00:55:21 Epoch7	loss: 0.665846
2017-09-21 00:55:30 Epoch

In [8]:
results

[{'params': {'final_act': 'tanh', 'hidden_act': 'tanh', 'loss': 'top1'},
  'results': (0.18717556111588304, 0.082711286194915232)},
 {'params': {'final_act': 'softmax', 'hidden_act': 'tanh', 'loss': 'top1'},
  'results': (0.008029278858722037, 0.002492041617197373)},
 {'params': {'final_act': 'linear', 'hidden_act': 'tanh', 'loss': 'top1'},
  'results': (0.16428277999775928, 0.068785676315704286)},
 {'params': {'final_act': 'relu', 'hidden_act': 'tanh', 'loss': 'top1'},
  'results': (0.078220114277178177, 0.030007142522288813)},
 {'params': {'final_act': 'softmax_logit',
   'hidden_act': 'tanh',
   'loss': 'top1'},
  'results': (0.0015685102886805842, 0.00037310501002944877)},
 {'params': {'final_act': 'leaky-0.001', 'hidden_act': 'tanh', 'loss': 'top1'},
  'results': (0.078574896366284494, 0.02938448420542766)},
 {'params': {'final_act': 'leaky-0.01', 'hidden_act': 'tanh', 'loss': 'top1'},
  'results': (0.082738917727900804, 0.030253965158118054)},
 {'params': {'final_act': 'leaky-0.1

In [44]:
result = pd.DataFrame(results)
result = result.join(pd.DataFrame(result["params"].to_dict()).T)[['error_during_train','failed','final_act','hidden_act','loss','results']]
tags = result['results'].apply(pd.Series)
result_name = ['recall', 'mrr']
tags = tags.rename(columns = lambda x : result_name[x])
dfNew = result.join(tags)[['error_during_train','failed','final_act','hidden_act','loss', 'recall', 'mrr']]
dfNew

,error_during_train,failed,final_act,hidden_act,loss,recall,mrr
0,NaN,NaN,tanh,tanh,top1,0.187176,0.082711
1,NaN,NaN,softmax,tanh,top1,0.008029,0.002492
2,NaN,NaN,linear,tanh,top1,0.164283,0.068786
3,NaN,NaN,relu,tanh,top1,0.078220,0.030007
4,NaN,NaN,softmax_logit,tanh,top1,0.001569,0.000373
5,NaN,NaN,leaky-0.001,tanh,top1,0.078575,0.029384
6,NaN,NaN,leaky-0.01,tanh,top1,0.082739,0.030254
7,NaN,NaN,leaky-0.1,tanh,top1,0.086529,0.033327
8,NaN,NaN,elu-1,tanh,top1,0.134108,0.054562
9,NaN,NaN,elu-0.1,tanh,top1,0.077753,0.028121


In [48]:
dfNew.sort_values(by=['recall', 'mrr'],ascending=False )

,error_during_train,failed,final_act,hidden_act,loss,recall,mrr
103,NaN,NaN,relu,relu,xe_logit,0.980487,0.980487
260,NaN,NaN,tanh,linear,bpr-max-1,0.409120,0.172560
248,NaN,NaN,elu-1,tanh,bpr-max-1,0.403257,0.163086
290,NaN,NaN,tanh,linear,bpr-max-10,0.388879,0.151644
268,NaN,NaN,elu-1,linear,bpr-max-1,0.387627,0.154828
270,NaN,NaN,tanh,tanh,bpr-max-10,0.386974,0.147847
242,NaN,NaN,linear,tanh,bpr-max-1,0.384229,0.156987
240,NaN,NaN,tanh,tanh,bpr-max-1,0.383930,0.163947
298,NaN,NaN,elu-1,linear,bpr-max-10,0.377600,0.136657
278,NaN,NaN,elu-1,tanh,bpr-max-10,0.377357,0.140827
